# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [ ]:
target: str = ''
start: str = '2025-01-01'
end: str = '2025-12-31'
fixed_date: str = 'false'

## Setup

Import some libraries

In [ ]:
import json
from datetime import date

import petl as etl
from utils.paths import PUBLISHED, SITE

Set the target directory, and ensure it exists

In [ ]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [ ]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end)
)

Write the timestamp file

In [ ]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end'], (str(x) for x in date_range))), f)

## Projects data

In [ ]:
projects = (
    etl
    .fromjson(PUBLISHED / 'programme/projects.json')
    .convert(['Date From', 'Date To'], etl.dateparser('%Y-%m-%d'))
    .selectge('Date To', date_range[0])
    .selectle('Date From', date_range[1])
)

In [ ]:
projects

In [ ]:
def rowgenerator(row):
    for v in row['Producing model (from Audiences Sync)']:
        yield [v]

producing_model = etl.cat(
    projects
    .rowmapmany(rowgenerator, header=['Producing model'])
    .aggregate('Producing model', len),

    projects
    .selecteq('Producing model (from Audiences Sync)', [])
    .aggregate(None, len)
    .addfield('Producing model', 'UNKNOWN')
)

In [ ]:
project_summary = {
    'count': projects.nrows(),
    'producing_model': dict(producing_model.records()),
    'names': list(projects.cut('Project Title').distinct().values('Project Title')),
}

In [ ]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        {
            'projects': project_summary
        },
        f,
        indent=2
    )

## Tickets

In [ ]:
ticketed_events = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
    .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
    .selectrangeopenleft('start', date_range[0], date_range[1])
    .selectcontains('eventType', 'MainProgrammeEvent')
)

In [ ]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(ticketed_events.cut('instance_id', 'event_name'))
    .cut('event_name', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
)

In [ ]:
ticket_summary = {
    'events': {
        'names': list(ticketed_events.cut('event_name').distinct().values('event_name')),
        'count': ticketed_events.nrows(),
    },
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
    }
}

In [ ]:
with open(TARGET / 'tickets.json', 'w') as f:
    json.dump(
        ticket_summary,
        f,
        indent=2
    )

## Volunteers

In [ ]:
shifts = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selecteq('type', 'BD25 Event')
    .selectrangeopenleft('date', date_range[0], date_range[1])
    .convertnumbers(strict=False)
)

In [ ]:
volunteer_shifts_summary = {
    'events': {
        'count': shifts.cut('rosterfy_event_name').distinct().nrows(),
        'names': list(shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    },
    'shifts': {
        'count': shifts.nrows(),
        'attended': sum(shifts.values('attended')),
        'hours': sum(shifts.values('hours')),
    }
}

In [ ]:
with open(TARGET / 'volunteer_shifts.json', 'w') as f:
    json.dump(
        volunteer_shifts_summary,
        f,
        indent=2
    )